In [1]:
import time
import requests
import os
import bs4
import requests
from selenium import webdriver

DRIVER_PATH = '/home/adithya/Downloads/chromedriver_linux64/chromedriver'

folder_name = './wildlife/'
def download_image(url, folder_name, num):
    # write image to file
    reponse = requests.get(url)
    if reponse.status_code==200:
        with open(os.path.join(folder_name, str(num)+".jpg"), 'wb') as file:
            file.write(reponse.content)


driver = webdriver.Chrome(executable_path=DRIVER_PATH)
search_URL = 'https://www.google.com/search?q=wildlife+photos&tbm=isch&ved=2ahUKEwifgIiK28j7AhViqXIEHZ7wA50Q2-cCegQIABAA&oq=wildli+photos&gs_lcp=CgNpbWcQARgAMgYIABAHEB4yBggAEAcQHjIGCAAQBxAeMgYIABAHEB4yBggAEAcQHjIGCAAQBxAeMgYIABAHEB4yBggAEAcQHjIGCAAQBxAeMgYIABAHEB46BAgAEEM6BQgAEIAEUPcHWMkhYOosaABwAHgAgAG7BIgBzwmSAQc0LjIuNS0xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=mWGAY9_PJOLSytMPnuGP6Ak&bih=948&biw=1920'
driver.get(search_URL)

driver.execute_script("window.scrollTo(0, 0);")
page_html = driver.page_source
pageSoup = bs4.BeautifulSoup(page_html, 'html.parser')

for i in range(1, 151):
    if i % 25 == 0:
        # to skip suggestions in goole
        continue

    xPath = """//*[@id="islrg"]/div[1]/div[%s]"""%(i)

    previewImageXPath = """//*[@id="islrg"]/div[1]/div[%s]/a[1]/div[1]/img"""%(i)
    previewImageElement = driver.find_element("xpath", previewImageXPath)
    previewImageURL = previewImageElement.get_attribute("src")

    driver.find_element("xpath", xPath).click()

    timeStarted = time.time()
    while True:
        imageElement = driver.find_element("xpath", """//*[@id="Sva75c"]/div/div/div/div[3]/div[2]/c-wiz/div[2]/div[1]/div[1]/div[2]/div/a/img""")
        imageURL= imageElement.get_attribute('src')

        if imageURL != previewImageURL: break
        else:  
            currentTime = time.time()
            if currentTime - timeStarted > 10:
                print("Timeout! Will download a lower resolution image and move onto the next one")
                break
    #Downloading image
    try:
        download_image(imageURL, folder_name, i)
        print("Downloaded element %s out of %s total. URL: %s" % (i, 151, imageURL))
    except:
        print("Couldn't download an image %s, continuing downloading the next one"%(i))

/tmp/ipykernel_12018/329624398.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


Timeout! Will download a lower resolution image and move onto the next one
Couldn't download an image 1, continuing downloading the next one
Couldn't download an image 2, continuing downloading the next one
Downloaded element 3 out of 151 total. URL: https://i.guim.co.uk/img/media/f75f84778dc6c20e519223cdd39ec0f974f2c9b0/0_239_3583_2150/master/3583.jpg?width=1200&height=900&quality=85&auto=format&fit=crop&s=679a9ea087d210112f70eda780c6fed1
Downloaded element 4 out of 151 total. URL: https://cdn.theatlantic.com/media/img/photo/2021/12/peoples-choice-wildlife-photographe/a01_MarcoGaiotti-1/original.jpg
Downloaded element 5 out of 151 total. URL: http://cdn.cnn.com/cnnnext/dam/assets/211130161931-26-wildlife-photographer-peoples-choice-shortlist-2021-scli-intl-gbr.jpg
Timeout! Will download a lower resolution image and move onto the next one
Couldn't download an image 6, continuing downloading the next one
Downloaded element 7 out of 151 total. URL: https://i.natgeofe.com/n/af4581b1-84a3-

In [ ]:
#from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import cv2
import sys
import numpy as np
from skimage import segmentation
import torch.nn.init

use_cuda = torch.cuda.is_available()
maxIter = 30
channels = 100
conv=2
minLabels = 3
visualize = False

class SegNet(nn.Module):
    def __init__(self,input_dim, nChannel, nConv):
        super(SegNet, self).__init__()
        self.nConv = nConv
        self.conv1 = nn.Conv2d(input_dim, 100, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(nChannel)
        self.conv2 = nn.ModuleList()
        self.bn2 = nn.ModuleList()
        for i in range(1):
            self.conv2.append( nn.Conv2d(100, 100, kernel_size=3, stride=1, padding=1))
            self.bn2.append( nn.BatchNorm2d(100))
        self.conv3 = nn.Conv2d(100, 100, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(nChannel)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.bn1(x)
        for i in range(1):
            x = self.conv2[i](x)
            x = F.relu(x)
            x = self.bn2[i](x)
        x = self.conv3(x)
        x = self.bn3(x)
        return x

def slic_feature_extraction(img):
    lab = segmentation.slic(img, compactness=100, n_segments=10000)
    lab = lab.reshape(img.shape[0]*img.shape[1])
    unique = np.unique(lab)
    slic_feat = []
    for i in range(len(unique)):
        slic_feat.append( np.where(lab == unique[i])[0])

    return slic_feat


def load_img(img_path):
    img = cv2.resize(cv2.imread(img_path), (500,500))
    data = torch.from_numpy(np.array([img.transpose((2, 0, 1)).astype('float32')/255.]))
    if use_cuda: data = data.cuda()
    data = Variable(data)

    return data, img

def train(data,img, slic_feat, out_path):

    model = SegNet(data.size(1), channels, conv)
    if use_cuda: model.cuda()

    model.train()

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
    label_colours = np.random.randint(255,size=(100,3))

    for batch_idx in range(maxIter):
        optimizer.zero_grad()
        
        output = model(data)[0]
        output = output.permute(1,2,0).contiguous().view(-1, channels)
        ignore, target = torch.max( output, 1 )
        im_target = target.data.cpu().numpy()
        nLabels = len(np.unique(im_target))
        
        if visualize:
            out_im = np.array([label_colours[ c % 100 ] for c in im_target])
            out_im = out_im.reshape(img.shape).astype( np.uint8 )
            cv2.imshow( "output", out_im )
            cv2.waitKey(10)

        # improving segmenration using superpixel
        for i in range(len(slic_feat)):
            labels_per_sp = im_target[slic_feat[i]]
            u_labels_per_sp = np.unique( labels_per_sp )
            hist = np.zeros(len(u_labels_per_sp))
            for j in range(len(hist)):
                hist[j] = len( np.where( labels_per_sp == u_labels_per_sp[j])[0] )
            im_target[slic_feat[i]] = u_labels_per_sp[np.argmax(hist)]
        target = torch.from_numpy( im_target )

        if use_cuda: target = target.cuda()

        target = Variable(target)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10== 0: print (batch_idx, '/', maxIter, ':', nLabels, loss.item())

        if nLabels <= minLabels:
            print ("nLabels", nLabels, "reached minLabels", minLabels, ".")
            break

    if not visualize:
        output = model(data)[0]
        output = output.permute(1,2,0).contiguous().view( -1, channels)
        ignore, target = torch.max(output, 1)
        im_target = target.data.cpu().numpy()
        out_im = np.array([label_colours[c % 100] for c in im_target])
        out_im = out_im.reshape(img.shape).astype(np.uint8)
    cv2.imwrite(out_path, out_im)


def unsuper_Seg(file_path, out_path):

    data, img = load_img(file_path)
    slic_feat = slic_feature_extraction(img)

    train(data, img, slic_feat, out_path)

In [8]:
folder = './demo_data'
out = './segmented_demo_data'

for file in os.listdir(folder):
    file_path = os.path.join(folder, file)
    out_path = os.path.join(out, file)
    print(file_path)

    unsuper_Seg(file_path, out_path)

./demo_data/adelina-tuca-profile.jpg
0 / 30 : 99 3.463144302368164
10 / 30 : 17 2.046722888946533
20 / 30 : 5 0.3239264190196991
./demo_data/wildlife.jpg
0 / 30 : 100 3.4706883430480957
10 / 30 : 30 1.6842962503433228
20 / 30 : 10 0.4387589395046234
./demo_data/nature_1.jpeg
0 / 30 : 100 3.765160083770752
10 / 30 : 36 2.1157238483428955
20 / 30 : 8 0.5479351282119751
./demo_data/nature_2.jpeg
0 / 30 : 100 3.785759210586548
10 / 30 : 27 1.5715562105178833
20 / 30 : 16 0.5876234173774719
./demo_data/dog.jpg
0 / 30 : 100 3.5409247875213623
10 / 30 : 43 2.3177425861358643
20 / 30 : 17 0.753579318523407
